In [7]:
import os 
import torch
import torchvision
import skimage
import numpy as np
import json

In [3]:
# for scene in ['vae5000/1e-4', 'vae5000/3e-4', 'vae5000/5e-5']:
#     for chk in [59, 79, 99]:
#         for axis_name in ['axis_x', 'axis_y', 'axis_z']:
#             directory = f"../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}" 
#             for scene_id in range(10):
#                 os.system(f"ffmpeg -r 6 -i {directory}/lightning_logs/version_0/crop_image/%04d_{scene_id:05d}.png -c:v libx264 -crf 12 -pix_fmt yuv420p {directory}_{scene_id:02d}.mp4")
                

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [14]:
def mark_brightness_color(img):
    img_gray = 0.299*img[..., 0]  + 0.587*img[..., 1] + 0.114*img[..., 2]
    highest_position = np.unravel_index(np.argmax(img_gray), img_gray.shape)
    highest_value = img_gray[highest_position]
    mask = np.abs(img_gray - highest_value) <= 1.0/255.0
    img[:, :, 0][mask] = 1
    img[:, :, 1][mask] = 0
    img[:, :, 2][mask] = 0
    return img

In [20]:
# create image size 11x2 
import skimage.transform


DATASET_DIR = "datasets/validation/rotate_point/axis_x"
#for scene in ['vae5000/1e-4', 'vae5000/3e-4', 'vae5000/5e-5']:
#    for chk in [59, 79, 99]:
for scene in ['vae5000/3e-4']:
    for chk in [179]:
        for axis_name in ['axis_x', 'axis_y', 'axis_z']:
            print(scene, chk, axis_name)
            for image_id in range(24):
                #rows = []
                row = []
                directory = f"../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}" 
                os.makedirs(f"{directory}/lightning_logs/version_0/compare", exist_ok=True)
                # load ground truth image
                gt_image = skimage.io.imread(f"../../datasets/validation/rotate_point/{axis_name}/images/{image_id:04d}.png")
                gt_image = skimage.img_as_float(gt_image)
                if gt_image.shape[-1] == 4:
                    # blend with black background
                    gt_image = gt_image[..., :3] * gt_image[..., 3:]# + (1 - gt_image[..., 3:])
                gt_image = skimage.transform.resize(gt_image, (256, 256))
                #gt_image = mark_brightness_color(gt_image)
                gt_image = torch.tensor(gt_image).permute(2, 0, 1)
                row.append(gt_image)
                for scene_id in range(10):
                    #img = torchvision.io.read_image(f"{directory}/lightning_logs/version_0/crop_image/{image_id:04d}_{scene_id:05d}.png") / 255.0
                    img = skimage.io.imread(f"{directory}/lightning_logs/version_0/crop_image/{image_id:04d}_{scene_id:05d}.png")
                    img = skimage.img_as_float(img)
                    img = skimage.transform.resize(img, (256, 256))
                    #img = mark_brightness_color(img)
                    img = torch.tensor(img).permute(2, 0, 1)
                    row.append(img)
                
                # load ground truth ball
                gt_ball = skimage.io.imread(f"../../datasets/validation/rotate_point/{axis_name}/ball/{image_id:04d}.png")
                gt_ball = skimage.img_as_float(gt_ball)
                if gt_ball.shape[-1] == 4:
                    # blend with black background
                    gt_ball = gt_ball[..., :3] * gt_ball[..., 3:]# + (1 - gt_ball[..., 3:])
                gt_ball = skimage.transform.resize(gt_ball, (256, 256))
                gt_ball = mark_brightness_color(gt_ball)
                gt_ball = torch.tensor(gt_ball).permute(2, 0, 1)
                row.append(gt_ball)
                
                # load gt image with torchvision
                for scene_id in range(10):
                    #img = torchvision.io.read_image(f"{directory}/lightning_logs/version_0/ball/{image_id:04d}_{scene_id:05d}.png") / 255.0
                    img = skimage.io.imread(f"{directory}/lightning_logs/version_0/ball/{image_id:04d}_{scene_id:05d}.png")
                    img = skimage.img_as_float(img)
                    img = skimage.transform.resize(img, (256, 256))
                    img = mark_brightness_color(img)
                    img = torch.tensor(img).permute(2, 0, 1)
                    row.append(img)

                image = torchvision.utils.make_grid(row, nrow=11)
                torchvision.utils.save_image(image, f"{directory}/lightning_logs/version_0/compare/{image_id:04d}.png")


vae5000/3e-4 179 axis_x
vae5000/3e-4 179 axis_y
vae5000/3e-4 179 axis_z


In [22]:
# create image size 11x2 
import skimage.transform


DATASET_DIR = "datasets/validation/rotate_point/axis_x"
# for scene in ['vae5000/1e-4', 'vae5000/5e-5']:
#     for chk in [59, 79, 99]:
#for scene in ['vae5000/3e-4']:
#    for chk in [159, 179, 199]:
        #for axis_name in ['light_x_minus', 'light_x_plus', 'light_y_minus', 'light_y_plus', 'light_z_minus', 'light_z_plus']:
for scene in ['vae5000/3e-4']:
   for chk in [179]:
        for axis_name in ['mask_topleft', 'mask_topright']:
            print(scene, chk, axis_name)
            with open(f'../../datasets/face/face2000_single/{axis_name}.json') as f:
                target_files = json.load(f)
            for image_count, target_file in enumerate(target_files):
                subdir, filename = target_file.split("/")
                if True:
                #for image_id in range(10):
                    row = []
                    #directory = f"../../output/20240703/val_axis/{scene}/chk{chk}/{axis_name}"
                    directory = f"../../output/20240703/val_axis_maskquater/{scene}/chk{chk}/{axis_name}" 
                    os.makedirs(f"{directory}/lightning_logs/version_0/compare", exist_ok=True)
                    # load ground truth image
                    gt_image = skimage.io.imread(f"../../datasets/face/face2000_single/images/{subdir}/{filename}.png")
                    gt_image = skimage.img_as_float(gt_image)
                    if gt_image.shape[-1] == 4:
                        # blend with black background
                        gt_image = gt_image[..., :3] * gt_image[..., 3:]# + (1 - gt_image[..., 3:])
                    gt_image = skimage.transform.resize(gt_image, (256, 256))
                    #gt_image = mark_brightness_color(gt_image)
                    gt_image = torch.tensor(gt_image).permute(2, 0, 1)
                    row.append(gt_image)
                    for scene_id in range(10):
                        img = skimage.io.imread(f"{directory}/lightning_logs/version_0/crop_image/{filename}_{scene_id:05d}.png")
                        img = skimage.img_as_float(img)
                        img = skimage.transform.resize(img, (256, 256))
                        img = torch.tensor(img).permute(2, 0, 1)
                        row.append(img)
                    
                    # load ground truth ball
                    gt_ball = skimage.io.imread(f"../../datasets/face/face2000_single/ball/{subdir}/{filename}_front.png")
                    gt_image = skimage.img_as_float(gt_image)
                    gt_ball = skimage.img_as_float(gt_ball)
                    if gt_ball.shape[-1] == 4:
                        # blend with black background
                        gt_ball = gt_ball[..., :3] * gt_ball[..., 3:]# + (1 - gt_ball[..., 3:])
                    gt_ball = skimage.transform.resize(gt_ball, (256, 256))
                    gt_ball = mark_brightness_color(gt_ball)
                    gt_ball = torch.tensor(gt_ball).permute(2, 0, 1)
                    row.append(gt_ball)
                    
                    # load gt image with torchvision
                    for scene_id in range(10):
                        #img = torchvision.io.read_image(f"{directory}/lightning_logs/version_0/ball/{image_id:04d}_{scene_id:05d}.png") / 255.0
                        img = skimage.io.imread(f"{directory}/lightning_logs/version_0/ball/{filename}_{scene_id:05d}.png")
                        img = skimage.img_as_float(img)
                        img = skimage.transform.resize(img, (256, 256))
                        img = mark_brightness_color(img)
                        img = torch.tensor(img).permute(2, 0, 1)
                        row.append(img)

                    image = torchvision.utils.make_grid(row, nrow=11)
                    torchvision.utils.save_image(image, f"{directory}/lightning_logs/version_0/compare/{image_count:04d}.png")


vae5000/3e-4 179 mask_topleft
vae5000/3e-4 179 mask_topright


In [19]:
for scene in ['vae5000/1e-4', 'vae5000/3e-4', 'vae5000/5e-5']:
    for chk in [59, 79, 99]:
      for axis_name in ['axis_x', 'axis_y', 'axis_z']:
        fname = f"../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}/compare.mp4"
        print(fname)
        if os.path.exists(fname):
            continue
        os.system(f'ffmpeg -r 6 -i "../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}/lightning_logs/version_0/compare/%04d.png" -c:v libx264 -crf 12 -pix_fmt yuv420p "../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}/compare.mp4"')  
    

../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk59/axis_x/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk59/axis_y/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk59/axis_z/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk79/axis_x/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk79/axis_y/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk79/axis_z/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk99/axis_x/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk99/axis_y/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/1e-4/chk99/axis_z/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk59/axis_x/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk59/axis_y/compare.mp4
../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk59/axis_z/comp

In [23]:
for scene in ['vae5000/3e-4']:
    for chk in [179]:
      for axis_name in ['axis_x', 'axis_y', 'axis_z']:
        fname = f"../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}/compare.mp4"
        print(fname)
        if os.path.exists(fname):
            continue
        os.system(f'ffmpeg -r 6 -i "../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}/lightning_logs/version_0/compare/%04d.png" -c:v libx264 -crf 12 -pix_fmt yuv420p "../../output/20240703/val_axis_lightdirection/{scene}/chk{chk}/{axis_name}/compare.mp4"')  
    

../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk179/axis_x/compare.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk179/axis_y/compare.mp4


[libx264 @ 0x5606f3d72280] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5606f3d72280] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x5606f3d72280] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=16 lookahead_threads=2 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=6 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=12.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk179/axis_y/compare.mp4':
  Metadata:
    encoder         : Lavf58.

../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk179/axis_z/compare.mp4


[libx264 @ 0x555b68fb2280] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x555b68fb2280] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x555b68fb2280] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=16 lookahead_threads=2 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=6 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=12.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '../../output/20240703/val_axis_lightdirection/vae5000/3e-4/chk179/axis_z/compare.mp4':
  Metadata:
    encoder         : Lavf58.